In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
import seaduck.lagrangian_budget as sdlb
import time

In [3]:
import os
path = '/sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/'
nt = len(os.listdir(path))
nt = 365

In [4]:
idate = 6999

In [5]:
files = sorted([path+i for i in os.listdir(path)],reverse = True)[:365]

In [6]:
neo = xr.open_zarr(files[0])

In [7]:
Np = len(neo.shapes)

In [8]:
from open4dense import *

In [9]:
wallt_path = '/sciserver/filedb09-01/ocean/wall_theta.zarr'
data_path = '/sciserver/filedb02-02/ocean/wenrui_temp/heat/'

In [10]:
ds = give_me_ecco_heat(data_path,wallt_path,time_mean_vel = True)

In [11]:
ds

<xarray.Dataset>
Dimensions:  (X: 90, Y: 90, Z: 50, face: 13, time: 9497, Xp1: 90, Yp1: 90,
              Zl: 50)
Coordinates: (12/31)
  * X        (X) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * Y        (Y) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * Z        (Z) float64 -5.0 -15.0 -25.0 ... -5.039e+03 -5.461e+03 -5.906e+03
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * time     (time) datetime64[ns] 1992-01-01T18:00:00 ... 2017-12-31T06:00:00
    CS       (face, Y, X) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    ...       ...
    dyG      (face, Y, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    maskS    (Z, face, Yp1, X) int8 dask.array<chunksize=(25, 7, 45, 45), meta=np.ndarray>
    maskW    (Z, face, Y, Xp1) int8 dask.array<chunksize=(25, 7, 45, 45), meta=np.ndarray>
    rAs      (face, Yp1, X) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    rAw      (face, Y, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    rAz      (face, Yp1, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
Data variables: (12/26)
    divumtm  (Z, face, Y, X) float64 dask.array<chunksize=(1, 13, 13, 90), meta=np.ndarray>
    divumtp  (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    divuptm  (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    divuptp  (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    divut    (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    tendH    (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ...       ...
    dif_v    (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    I        (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    lhs      (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    txprime  (time, Z, face, Y, Xp1) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    typrime  (time, Z, face, Yp1, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    tzprime  (time, Zl, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
Attributes: (12/62)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on the curvilinear lat-l...
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    time_coverage_duration:          P1D
    time_coverage_end:               1992-01-02T00:00:00
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-01-01T12:00:00
    title:                           ECCO Ocean Temperature and Salinity - Da...
    uuid:                            c2efe382-4168-11eb-8956-0cc47a3f480f

In [12]:
# close = ds['R'] - (ds['A']+ds['F']+ds['I']+ds['e_ua']+ds['E']+ds['lhs']+ds['dif_h']+ds['dif_v'])

# def show_close_notime(ds_slc):
#     fig,axes = plt.subplots(2,2)
#     axes = axes.ravel()
#     count = 0
#     for iz in [0,5]:
#         for face in [1,10]:
#             print(ds_slc.isel(Z = iz,face = face).max().values)
#             axes[count].imshow(ds_slc.isel(Z = iz,face = face))
#             count+=1
#     plt.show()

# def show_two_snap(close):
#     show_close_notime(close.isel(time = 1))
#     show_close_notime(close.isel(time = -100))

# show_two_snap(close)

In [13]:
rhs_list = ['e_ua','E','dif_h','dif_v','A','I','F']
termlist = rhs_list+['lhs']

In [14]:
dataset_date_id = np.array(ds.time[idate:idate-nt:-1])
to_isel = np.arange(idate,idate-nt,-1)

In [15]:
hovmoller = xr.Dataset()
for var in rhs_list+['lhs', 'sf', 'sl', 'lon', 'lat', 'dep']:
    hovmoller[var] = xr.DataArray(np.zeros((len(dataset_date_id),Np)),dims = ('time','space'))

In [16]:
def cumu_map_zarr(neo, contr_dic, varname, last):
    value = contr_dic[varname]
    ind = tuple(np.array(i)[:-1] for i in [neo.iz-1, neo.face, neo.iy, neo.ix])
    array = np.zeros((50,13,90,90))
    np.add.at(array, ind, value)
    return array

def cumu_map_array(neo, value):
    ind = tuple(np.array(i)[:-1] for i in [neo.iz-1, neo.face, neo.iy, neo.ix])
    array = np.zeros((50,13,90,90))
    np.add.at(array, ind, value)
    return array

maps = {}
for var in rhs_list+['count']:
    maps[var] = np.zeros((50,13,90,90))

In [17]:
t1 = time.time()
for it,(dataset_date, particle_name) in enumerate(zip(to_isel,files)):
    my = ds.isel(time = dataset_date)
    prefetch_term = sdlb.prefetch_scalar(my, termlist)
    walls_array = sdlb.prefetch_vector(my,xname="txprime", yname="typrime", zname="tzprime")
    if it %10==0:
        print('dataset_date:', particle_name, 
              str(it)+'/'+str(nt), 
              time.time()-t1)
    neo = xr.open_zarr(particle_name)
    ind1 = tuple(neo['ind1'].values)
    ind2 = tuple(neo['ind2'].values)
    
    frac = np.array(neo.frac)
    shapes = np.array(neo.shapes)
    tres = np.array(neo.tres)
    tact = np.array(neo.tt)
    
    ind = tuple(np.array(i) for i in [neo.iz-1,neo.face,neo.iy,neo.ix])
    
    step_dic = sdlb.read_prefetched_scalar(ind,termlist,prefetch_term)
    first, last = sdlb.first_last_neither(shapes,return_neither = False)
    s1 = walls_array[ind1]
    s2 = walls_array[ind2]
    
    s_wall = s1*frac + (1-frac)*s2
    nstep = len(frac)-1
    deltas = np.nan_to_num(np.diff(s_wall))
    deltas[last[:-1]] = 0
    tres_used = -tres[1:]
    tres_used[last[:-1]] = 0 
    
    correction = sdlb.lhs_contribution(tact, step_dic, last)
    rhs_contr = deltas - correction
    
    contr_dic = sdlb.contr_p_relaxed(rhs_contr, tres_used, step_dic, rhs_list)
    contr_dic['lhs'] = correction

    for var in rhs_list+['lhs']:
        cumsum = np.cumsum(np.nan_to_num(contr_dic[var]))
        cumsum = np.insert(cumsum[last-1],0,0)
        hovmoller[var][it,:] = np.diff(cumsum)
        # assert np.isclose(np.nansum(hovmoller[var][it]),np.nansum(contr_dic[var]))
    hovmoller['sl'][it,:] = s_wall[last]
    hovmoller['sf'][it,:] = s_wall[first]
    hovmoller['lon'][it,:] = np.array(neo.xx[first])
    hovmoller['lat'][it,:] = np.array(neo.yy[first])
    hovmoller['dep'][it,:] = np.array(neo.zz[first])
    for var in rhs_list:
        maps[var] += cumu_map_zarr(neo, contr_dic, var, last)
    maps['count'] += cumu_map_array(neo, np.ones_like(tres_used))

dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2011-02-28T00:00:00 0/365 10.800268411636353
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2011-02-18T00:00:00 10/365 104.90749740600586
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2011-02-08T00:00:00 20/365 200.18371748924255
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2011-01-29T00:00:00 30/365 295.9154782295227
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2011-01-19T00:00:00 40/365 390.0550184249878
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2011-01-09T00:00:00 50/365 484.3424379825592
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/nc_new/Seed2011_2010-12-30T00:00:00 60/365 577.8828957080841
dataset_date: /sciserver/filedb04-01/ocean/wenrui_temp/particle_fil

In [18]:
maps_ds = xr.Dataset()
for var in rhs_list+['count']:
    maps_ds[var] = xr.DataArray(maps[var],dims = ('Z','face','Y','X'))

In [19]:
# maps_ds = xr.Dataset()
# for var in rhs_list+['count']:
#     maps_ds[var] = xr.DataArray(maps[var],dims = ('Z','face','Y','X'))
nep_path = '/sciserver/filedb04-01/ocean/wenrui_temp/particle_file/WA/'
map_path = nep_path+ 'map1y.zarr'
table_path = nep_path+'table1y.zarr'
maps_ds.to_zarr(map_path, mode = 'w')
hovmoller.to_zarr(table_path, mode = 'w')
print('success')

success


In [20]:
maps_ds

<xarray.Dataset>
Dimensions:  (Z: 50, face: 13, Y: 90, X: 90)
Dimensions without coordinates: Z, face, Y, X
Data variables:
    e_ua     (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    E        (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dif_h    (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dif_v    (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    A        (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    I        (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    F        (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    count    (Z, face, Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [21]:
hovmoller

<xarray.Dataset>
Dimensions:  (time: 365, space: 10009)
Dimensions without coordinates: time, space
Data variables: (12/13)
    e_ua     (time, space) float64 -0.002535 -0.001203 ... 0.0003461 -0.000175
    E        (time, space) float64 -0.03075 -0.01508 ... 0.02159 0.01501
    dif_h    (time, space) float64 0.00366 0.001893 ... -0.0009103 -0.002941
    dif_v    (time, space) float64 0.3964 0.1703 0.2001 ... 0.004553 0.0003464
    A        (time, space) float64 -0.06045 -0.03179 ... -0.1193 -0.06313
    I        (time, space) float64 -0.3682 -0.2409 ... -0.002231 0.0008168
    ...       ...
    lhs      (time, space) float64 0.1347 0.149 0.149 ... 0.0007461 0.01017
    sf       (time, space) float64 3.953 3.929 3.985 ... 0.4833 0.5109 1.031
    sl       (time, space) float64 4.018 3.957 4.026 ... 0.4821 0.4156 0.9916
    lon      (time, space) float64 108.9 108.5 108.4 108.8 ... 121.7 118.8 121.0
    lat      (time, space) float64 -34.53 -34.31 -34.16 ... -8.033 -3.848 -5.788
    dep      (time, space) float64 -0.8511 -9.49 -2.831 ... -34.43 -18.15 -25.02